In [1]:
import hashlib as hl
import csv
from datetime import datetime
from os import close
import numpy as np
import logging
import argparse
import sys
from datetime import datetime
from datetime import timezone
import socket
import math
import pickle
import random
import pandas as pd
#from Functions.Functions_wireshark import *

from Functions.Optimized_Parser_Labelling import *
log = logging.getLogger()
log.setLevel(logging.DEBUG)

from Functions.Optimized_Parser_Labelling import pcap_parser

In [2]:
from datetime import datetime, timezone, timedelta

df_p=pd.read_csv('./UNSW-NB15_preprocessed.csv')
df_p.rename(columns={'proto':'protocol_m'}, inplace=True)

# Convert UNIX timestamp to datetime, adjust time, and convert back to UNIX
def adjust_time(unix_time):
    dt = datetime.fromtimestamp(unix_time, tz=timezone.utc)
    if 1 <= dt.hour <= 7:
        dt += timedelta(hours=15)
    else:
        dt += timedelta(hours=3)
    return int(dt.timestamp())

df_p['stime'] = df_p['stime'].apply(adjust_time)

df_p=df_p.sort_values(by='stime')

C:\Users\steph\PycharmProjects\payloadbyte\.venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
## Labeling & Combining of Parsed PCAP Files

In [4]:
import os
import pandas as pd
import logging

def combine_UNSW(in_folder_path, out_path):
    # Initialize empty DataFrame with expected columns
    combine = pd.DataFrame(columns=[
        'srcip', 'sport', 'dstip', 'dsport', 'protocol_m',
        'sttl', 'total_len', 'payload', 't_delta', 'stime', 'label'
    ])

    # Discover all labelled CSV files
    for root, dirs, files in os.walk(in_folder_path):
        for file in files:
            if file.startswith('labelled_') and file.endswith('.csv'):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                combine=combine.append(df, ignore_index=True)
                print(combine.shape)

    # Save combined result
    os.makedirs(out_path, exist_ok=True)
    csv_out = os.path.join(out_path, "combined_labelled_pcap_csv.csv")
    logging.info("Exporting combined CSV file....")
    combine.to_csv(csv_out, index=False)
    return combine


In [5]:
label_csv_folder = './UNSW_results/Labelled_pcap_file/'
out_combined_path = './UNSW_results/Labelled_pcap_file/Combined/'

df_payload = combine_UNSW(label_csv_folder, out_combined_path)


Added: ./UNSW_results/Labelled_pcap_file/17-2-1015\labelled_pcap_csv_1.csv, shape now: (148653, 17)
Added: ./UNSW_results/Labelled_pcap_file/17-2-1015\labelled_pcap_csv_10.csv, shape now: (547786, 17)
Added: ./UNSW_results/Labelled_pcap_file/17-2-1015\labelled_pcap_csv_11.csv, shape now: (897933, 17)
Added: ./UNSW_results/Labelled_pcap_file/17-2-1015\labelled_pcap_csv_2.csv, shape now: (1231169, 17)
Added: ./UNSW_results/Labelled_pcap_file/17-2-1015\labelled_pcap_csv_3.csv, shape now: (1231169, 17)
Added: ./UNSW_results/Labelled_pcap_file/17-2-1015\labelled_pcap_csv_4.csv, shape now: (1614591, 17)
Added: ./UNSW_results/Labelled_pcap_file/17-2-1015\labelled_pcap_csv_5.csv, shape now: (1999416, 17)
Added: ./UNSW_results/Labelled_pcap_file/17-2-1015\labelled_pcap_csv_6.csv, shape now: (2378888, 17)
Added: ./UNSW_results/Labelled_pcap_file/17-2-1015\labelled_pcap_csv_7.csv, shape now: (2805794, 17)
Added: ./UNSW_results/Labelled_pcap_file/17-2-1015\labelled_pcap_csv_8.csv, shape now: (3158

INFO:root:Exporting combined CSV file....


In [6]:
df_payload.shape

(7971924, 17)

In [7]:
df_payload.drop_duplicates(inplace=True)

In [8]:
df_payload.label.value_counts()

0    7794035
1      37134
Name: label, dtype: int64

In [9]:
df_payload.drop(df_payload[df_payload.payload.isnull()].index,inplace=True)

In [10]:
df_payload.shape

(2623629, 17)

In [11]:
x=df_payload['payload']
new=[]
for p in range(len(x)):
    try:
        o=(int((x.iloc[p]), 16))
        if o>0:
            new.append(1)
        else:
            new.append(0)
    except Exception as e:
        print(p)
        new.append('Big')

In [12]:
df_payload['payload_int']=new

In [13]:
df_payload.drop(df_payload[df_payload.payload_int==0].index,inplace=True)

In [14]:
## After Removing zero payload
df_payload.label.value_counts()

0    2604389
1      18928
Name: label, dtype: int64

In [15]:
df_payload.pop('payload_int')

0          1
1          1
2          1
12         1
17         1
          ..
7971919    1
7971920    1
7971921    1
7971922    1
7971923    1
Name: payload_int, Length: 2623317, dtype: int64

In [16]:
df_payload.sttl=df_payload.sttl.astype('int32')
df_payload.dsport=df_payload.dsport.astype('int32')
df_payload.sport=df_payload.sport.astype('int32')
df_payload.total_len=df_payload.total_len.astype('int32')

In [17]:
df_payload.to_csv(r'./UNSW_results\Labelled_pcap_file\Combined\combined_labelled_cleaned_sorted_pcap_csv.csv',index=False)

In [3]:
df_payload= pd.read_csv(r'./UNSW_results\Labelled_pcap_file\Combined\combined_labelled_cleaned_sorted_pcap_csv.csv')

In [4]:
df_payload.iloc[:,:-1]

,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,payload,t_delta,stime,label,frame_num,protocol_s,ltime,proto,dur
0,175.45.176.3,22592,149.171.126.16,143,tcp,62,61,613030332053454c4543542022494e424f58220d0a,0.000041,1424219008,0,2.0,imap,1.424219e+09,tcp,0.649902
1,175.45.176.3,22592,149.171.126.16,143,tcp,62,68,6130303420464554434820313a2a202855494420464c41...,0.000010,1424219008,0,8.0,imap,1.424219e+09,tcp,0.649902
2,175.45.176.3,22592,149.171.126.16,143,tcp,62,66,61303035204645544348203120424f44592e5045454b5b...,0.000009,1424219008,0,26.0,imap,1.424219e+09,tcp,0.649902
3,175.45.176.3,62994,149.171.126.14,179,tcp,254,85,ffffffffffffffffffffffffffffffff002d01044eea00...,0.000009,1424219009,0,98.0,bgp,1.424219e+09,tcp,0.449454
4,175.45.176.3,63888,149.171.126.14,179,tcp,254,85,ffffffffffffffffffffffffffffffff002d01044eea00...,0.000009,1424219011,0,198.0,bgp,1.424219e+09,tcp,0.258687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623312,149.171.126.5,62109,59.166.0.5,62203,tcp,29,1352,3c4de90d01a5a3f0820fa59aa68e483f6c1d7c9737baa3...,0.000011,1421948286,0,1818409.0,6,1.421948e+09,tcp,0.000000
2623313,149.171.126.5,62109,59.166.0.5,62203,tcp,29,1352,df67256125617a775bbe4188aca717837f3fd4af07397e...,0.000009,1421948286,0,1818425.0,6,1.421948e+09,tcp,0.000000
2623314,149.171.126.5,62109,59.166.0.5,62203,tcp,29,1352,d561ac50ab8f88f976d01da2846ee943d685db14672be2...,0.000012,1421948286,0,1818437.0,6,1.421948e+09,tcp,0.000000
2623315,149.171.126.5,62109,59.166.0.5,62203,tcp,29,1352,8e3696fdb6ffd43d51506e64c6f4fdfd8caa8fd4415df6...,0.000009,1421948286,0,1818451.0,6,1.421948e+09,tcp,0.000000


In [5]:
from imblearn.under_sampling import RandomUnderSampler 
df_payload.attack_cat.value_counts()

normal            2604389
generic              5355
exploits             4567
fuzzers              3984
reconnaissance       2584
dos                  1156
backdoor              478
analysis              408
shellcode             364
worms                  32
Name: attack_cat, dtype: int64

In [6]:
# We can either do binary classification on 'label' (normal vs attack)
# Or, we can do multi-class classification on 'attack_cat' (10 types of attacks)

df_payload.label.value_counts()

0    2604389
1      18928
Name: label, dtype: int64

In [7]:
# We will do multi-class classification.

df_payload.drop(columns=['label'], inplace=True)
df_payload.rename(columns={'attack_cat': 'label'}, inplace=True)

In [8]:
df_payload.label.value_counts()

normal            2604389
generic              5355
exploits             4567
fuzzers              3984
reconnaissance       2584
dos                  1156
backdoor              478
analysis              408
shellcode             364
worms                  32
Name: label, dtype: int64

In [9]:

dict1={ 'normal': 50000
}

# We are currently unable to allocate enough memory with 2.6m instances of normal; downsample to 50k.

In [10]:
rus = RandomUnderSampler(random_state=42,sampling_strategy=dict1)

In [11]:
X_res, y_res = rus.fit_resample(df_payload.iloc[:,:-1], df_payload.iloc[:,-1])

In [12]:
X_res['label']=y_res

In [13]:
X_res.to_csv(r'./CICIDS_results\Labelled_pcap_file\Combined\combined_labelled_cleaned_undersampled_pcap_csv.csv',index=False)

In [14]:
X_res = pd.read_csv('./CICIDS_results/Labelled_pcap_file/Combined/combined_labelled_cleaned_undersampled_pcap_csv.csv')
y_res = X_res['label']

In [16]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.preprocessing import LabelEncoder
from Functions.Pipeline import *

In [17]:
## Transformation into Byte-wise features

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(X_res['protocol_m'])
X_res['protocol']=label

X_res.drop("protocol_m", axis=1, inplace=True)
X_res.drop("protocol_s", axis=1, inplace=True)
X_res.drop("frame_num", axis=1, inplace=True)

# X_res['attack_cat']=y_res
X_res['label']=y_res

In [19]:
X_tr,Ytrain =payload_to_bytes(X_res,1500)

In [20]:
X_res.iloc[:, :-1]

,srcip,sport,dstip,dsport,sttl,total_len,payload,t_delta,stime,ltime,proto,dur,label
0,175.45.176.1,0,149.171.126.17,0,254,52,010000058119d5fa0001c3a923bd1ab790d07bd5,0.000009,1424219068,1.424219e+09,sctp,0.000009,analysis
1,175.45.176.3,0,149.171.126.14,0,254,78,000108000000000000ffffff0000003401030400040480...,0.000009,1424219101,1.424219e+09,gre,0.000009,analysis
2,175.45.176.1,0,149.171.126.17,0,254,100,4250532120363720425053212036372042505321203637...,0.000006,1424244396,1.424244e+09,pim,0.000006,analysis
3,175.45.176.1,0,149.171.126.17,0,254,100,4250532120363820425053212036382042505321203638...,0.000009,1424244397,1.424244e+09,others,0.000003,analysis
4,175.45.176.1,0,149.171.126.17,0,254,100,4250532120363820425053212036382042505321203638...,0.000009,1424244397,1.424244e+09,others,0.000009,analysis
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68923,175.45.176.2,26948,149.171.126.15,80,254,1500,474554202f64656661756c742e6964613f4e4e4e4e4e4e...,0.000007,1424244177,1.424244e+09,tcp,0.282141,worms
68924,175.45.176.2,26948,149.171.126.15,80,254,1500,feffff69d28d66f050899574feffff8b45088b8d50feff...,0.000005,1424244177,1.424244e+09,tcp,0.282141,worms
68925,175.45.176.2,26948,149.171.126.15,80,254,1159,854cfeffff8b8d64feffff0fbe1185d27402ebd38bf46a...,0.000002,1424244177,1.424244e+09,tcp,0.282141,worms
68926,175.45.176.3,55185,149.171.126.11,80,254,329,474554202f6367692d62696e2f617773746174732f6177...,0.000010,1424244273,1.424244e+09,tcp,0.469675,worms


In [21]:
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,0])))    # srcip
print('Done with srcip')
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,1])))    # sport
print('Done with sport')
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,2])))    # dstip
print('Done with dstip')
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,3])))    # dsport
print('Done with dsport')
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,4])))    # sttl
print('Done with sttl')
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,5])))    # total_len 
print('Done with total_len')
# SKIP PAYLOAD
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,7])))    # t_delta
print('Done with t_delta')
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,8])))    # stime
print('Done with stime')
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,10])))   # SKIP ltime; go to proto
print('Done with proto')

Done with srcip
Done with sport
Done with dstip
Done with dsport
Done with sttl
Done with total_len
Done with t_delta
Done with stime
Done with proto


In [22]:
name=[]
for x in range(1,1501):
    name.append("payload_byte_"+str(x))
name.append("srcip")
name.append("srcport")
name.append("dstip")
name.append("dstport")
name.append("ttl")
name.append("total_len")
name.append("t_delta")
name.append("Timestamp")
name.append("protocol_m")

In [23]:
df = pd.DataFrame(X_tr, columns=name)
df['label']=Ytrain

In [24]:
df.iloc[:,:-1]

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,payload_byte_1500,srcip,srcport,dstip,dstport,ttl,total_len,t_delta,Timestamp,protocol_m
0,1,0,0,5,129,25,213,250,0,1,...,0,175.45.176.1,0,149.171.126.17,0,254,52,0.000009,1424219068,sctp
1,0,1,8,0,0,0,0,0,0,255,...,0,175.45.176.3,0,149.171.126.14,0,254,78,0.000009,1424219101,gre
2,66,80,83,33,32,54,55,32,66,80,...,0,175.45.176.1,0,149.171.126.17,0,254,100,0.000006,1424244396,pim
3,66,80,83,33,32,54,56,32,66,80,...,0,175.45.176.1,0,149.171.126.17,0,254,100,0.000009,1424244397,others
4,66,80,83,33,32,54,56,32,66,80,...,0,175.45.176.1,0,149.171.126.17,0,254,100,0.000009,1424244397,others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68923,71,69,84,32,47,100,101,102,97,117,...,0,175.45.176.2,26948,149.171.126.15,80,254,1500,0.000007,1424244177,tcp
68924,254,255,255,105,210,141,102,240,80,137,...,0,175.45.176.2,26948,149.171.126.15,80,254,1500,0.000005,1424244177,tcp
68925,133,76,254,255,255,139,141,100,254,255,...,0,175.45.176.2,26948,149.171.126.15,80,254,1159,0.000002,1424244177,tcp
68926,71,69,84,32,47,99,103,105,45,98,...,0,175.45.176.3,55185,149.171.126.11,80,254,329,0.00001,1424244273,tcp


In [25]:
df.to_csv(r'./UNSW_results\Converted_data_UNSW_final.csv',index=False)

In [ ]:
## Miniflow Files

In [26]:
aa = pd.read_csv('./UNSW_results\Converted_data_UNSW_final.csv')

In [27]:
grouped_df = aa.groupby(['srcip','srcport','dstip','dstport','protocol_m'], as_index=False)
i=0

os.makedirs(f'./UNSW_results\miniflow/', exist_ok=True)

for key, item in grouped_df:
    item.to_csv(f'./UNSW_results/miniflow/miniflow-{i}.csv', index=False)
    i +=1

In [ ]:
## Packet File Generation

In [3]:
# df = pd.read_csv('./CICIDS_results/miniflow/miniflow-0.csv')

In [4]:
# print(df.head)

<bound method NDFrame.head of    payload_byte_1  payload_byte_2  payload_byte_3  payload_byte_4  \
0              67             194             216             240   
1             111             246              23               4   

   payload_byte_5  payload_byte_6  payload_byte_7  payload_byte_8  \
0             181             111             186             187   
1              33             158             156             127   

   payload_byte_9  payload_byte_10  ...        srcip  srcport          dstip  \
0             238              109  ...  1.226.51.14       80  192.168.10.12   
1             236              226  ...  1.226.51.14       80  192.168.10.12   

   dstport  protocol_m  ttl  total_len   t_delta   Timestamp   label  
0    55896         tcp   48       1500  0.000001  1499186460  BENIGN  
1    55896         tcp   48       1500  0.000000  1499186460  BENIGN  

[2 rows x 1510 columns]>


In [29]:
import os
import pandas as pd

folder_path = r'./UNSW_results\miniflow' # Replace with the path to your folder
output_path = r'./UNSW_results\packets-7' # Replace with the path to your output folder
i = 1

os.makedirs(f'./UNSW_results\packets-7/', exist_ok=True)

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        if len(df) >= 7:
            output_filename = 'packet-{}.csv'.format(i)
            output_file_path = os.path.join(output_path, output_filename)
            df = df.sort_values(by=['Timestamp'])
            selected_df = df.head(7)
            selected_df.to_csv(output_file_path, index=False)
            i += 1

In [ ]:
## Converting Packets to Flows

In [30]:
import pandas as pd
import glob

dfs = []
for file in glob.glob('./UNSW_results/packets-7/*.csv'):
    df = pd.read_csv(file)
    dfs.append(df)

all_df = pd.concat(dfs, ignore_index=True)

In [43]:
all_df = pd.concat(dfs, ignore_index=True)

In [44]:
all_df

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,srcip,srcport,dstip,dstport,ttl,total_len,t_delta,Timestamp,protocol_m,label
0,152,148,172,31,179,60,61,127,82,76,...,10.40.170.2,0,10.40.170.2,0,0,52,0.000014,1421927955,arp,normal
1,36,194,0,27,23,5,158,28,8,0,...,10.40.170.2,0,10.40.170.2,0,0,1500,0.000012,1421936595,arp,normal
2,152,162,174,122,224,228,63,177,215,142,...,10.40.170.2,0,10.40.170.2,0,0,1352,0.000019,1421938515,arp,normal
3,36,194,0,27,23,5,158,28,8,0,...,10.40.170.2,0,10.40.170.2,0,0,1500,0.000018,1421942295,arp,normal
4,152,164,174,105,45,63,64,125,33,184,...,10.40.170.2,0,10.40.170.2,0,0,1500,0.000015,1421942835,arp,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,66,80,83,33,32,99,32,66,80,83,...,175.45.176.1,0,149.171.126.17,0,254,90,0.000008,1424244339,pup,dos
1571,66,80,83,33,32,99,32,66,80,83,...,175.45.176.1,0,149.171.126.17,0,254,90,0.000008,1424244339,pup,exploits
1572,66,80,83,33,32,99,32,66,80,83,...,175.45.176.1,0,149.171.126.17,0,254,90,0.000008,1424244339,pup,fuzzers
1573,66,80,83,33,32,99,32,66,80,83,...,175.45.176.1,0,149.171.126.17,0,254,90,0.000008,1424244339,pup,generic


In [45]:
print(df[['srcip', 'srcport', 'dstip', 'dstport']].isnull().sum())

srcip      0
srcport    0
dstip      0
dstport    0
dtype: int64


In [46]:
unique_protocols = all_df['protocol_m'].unique()

In [47]:
unique_protocols

array(['arp', 'tcp', 'rdp', 'rsvp', 'sccopmce', 'sctp', 'secure-vmtp',
       'sep', 'snp', 'sps', 'sun-nd', 'swipe', 'unas', 'vmtp', 'ospf',
       'udp', 'gre', 'ax.25', 'crtp', 'crudp', 'dgp', 'egp', 'emcon',
       'etherip', 'fire', 'gmtp', 'hmp', 'ib', 'ip', 'ipip', 'iplt',
       'ipv6', 'micp', 'mobile', 'nvp', 'others', 'pim', 'pipe', 'pnni',
       'pup'], dtype=object)

In [48]:

protocol_map = {
    'icmp': 1,
    'igmp': 2,
    'ipip': 4,       # IP in IP encapsulation
    'tcp': 6,
    'udp': 17,
    'gre': 47,
    'esp': 50,
    'ah': 51,
    'egp': 8,
    'ospf': 89,
    'sctp': 132,
    'rsvp': 46,
    'pim': 103,
    'ax.25': 93,
    'ip': 0,
    'ipv6': 41,
    'vmtp': 81,
    'sun-nd': 77,
    'dgp': 86,
    'hmp': 20,
    'nvp': 11,
    'snp': 109,
    'sps': 83,
    'unas': 106,
    'rdp': 27,
    'crtp': 126,
    'crudp': 127,
    'sep': 33,
    'pipe': 131,
    'pnni': 102,
    'mobile': 55,
    'fire': 83,
    'micp': 95,
    'secure-vmtp': 92,
    'iplt': 129,
    'ib': 114,
    'etherip': 97,
    'others': -1,      # dataset-specific
    'emcon': 14,
    'pup': 12,
    'sccopmce': 128,     # not in IANA list , but sscopmce is -- assume typo.
    'arp': -1           # not in IANA list  - treat as 'other'
}

all_df['protocol_m'] = all_df['protocol_m'].map(protocol_map)

In [49]:
all_df['protocol_m']

0       -1.0
1       -1.0
2       -1.0
3       -1.0
4       -1.0
        ... 
1570    12.0
1571    12.0
1572    12.0
1573    12.0
1574    12.0
Name: protocol_m, Length: 1575, dtype: float64

In [50]:
def create_flow_id(row):
    if row['label'] == 'BENIGN':
        return row['dstip'] + '-' + row['srcip'] + '-' + str(row['dstport']) + '-' + str(row['srcport']) + '-' + str(row['protocol_m'])
    else:
        return row['srcip'] + '-' + row['dstip'] + '-' + str(row['srcport']) + '-' + str(row['dstport']) + '-' + str(row['protocol_m'])
        
all_df['flow_id'] = all_df.apply(create_flow_id, axis=1)

In [51]:
all_df['flow_id'].nunique()

221

In [52]:
all_df['flow_id']

0           10.40.170.2-10.40.170.2-0-0--1.0
1           10.40.170.2-10.40.170.2-0-0--1.0
2           10.40.170.2-10.40.170.2-0-0--1.0
3           10.40.170.2-10.40.170.2-0-0--1.0
4           10.40.170.2-10.40.170.2-0-0--1.0
                        ...                 
1570    175.45.176.1-149.171.126.17-0-0-12.0
1571    175.45.176.1-149.171.126.17-0-0-12.0
1572    175.45.176.1-149.171.126.17-0-0-12.0
1573    175.45.176.1-149.171.126.17-0-0-12.0
1574    175.45.176.1-149.171.126.17-0-0-12.0
Name: flow_id, Length: 1575, dtype: object

In [53]:
import pandas as pd

# create a list of unique flow IDs
flow_ids = all_df['flow_id'].unique()

# create a dataframe with columns for flow number and flow ID
df = pd.DataFrame({'flow_number': range(1, len(flow_ids)+1),
                   'flow_id': flow_ids})

# save the dataframe to a CSV file
df.to_csv('UNSW-7-number-packets-flow_ids.csv', index=False)


In [54]:
import pandas as pd

# group packets by their flow_id
groups = all_df.groupby('flow_id')

In [55]:
# initialize an empty dataframe to store the extracted flow information
new_df = pd.DataFrame()

In [56]:
import pandas as pd

# define a function to extract flow information
def extract_flow_info(group):
    # take the mean of all three packet feature values for this flow
    flow_features = group.drop(['flow_id','srcip', 'srcport', 'dstip', 'dstport', 'Timestamp', 'label','protocol_m'], axis=1).mean(numeric_only=True)
    
    # extract the protocol of the flow
    flow_protocol = group['protocol_m'].unique()[0]
    
    # extract the label of the flow
    flow_label = group['label'].unique()[0]
    
    # return a new row with the mean features, the flow protocol, and the flow label
    return pd.concat([flow_features, pd.Series(flow_protocol), pd.Series(flow_label)])

# group packets by their flow_id and apply the extract_flow_info function to each group
new_df = all_df.groupby('flow_id').apply(extract_flow_info).reset_index(drop=True)
print("Grouped packets by flow_id and extracted flow information...")
# rename the protocol and label columns
#new_df = new_df.rename(columns={0: 'protocol_m', 1: 'label'})

# save the new dataframe to a new csv file
new_df.to_csv('UNSW-7-number-packets-to-flows.csv', index=False)
print("Saved the new dataframe to csv file...")

Grouped packets by flow_id and extracted flow information...
Saved the new dataframe to csv file...


In [58]:
import pandas as pd

df = pd.read_csv('./UNSW-7-number-packets-to-flows.csv')
print(df.head())


   payload_byte_1  payload_byte_2  payload_byte_3  payload_byte_4  \
0      102.285714      174.000000       99.142857            66.0   
1       82.714286      131.142857       51.000000            82.0   
2       31.285714      106.571429      129.000000           131.0   
3      112.571429        0.428571      129.000000           131.0   
4       18.571429      138.142857      129.000000           131.0   

   payload_byte_5  payload_byte_6  payload_byte_7  payload_byte_8  \
0       82.714286       79.142857      103.571429       98.571429   
1       54.714286       97.285714       74.571429       96.714286   
2        0.000000        1.000000        0.000000        0.000000   
3        0.000000        1.000000        0.000000        0.000000   
4        0.000000        1.000000        0.000000        0.000000   

   payload_byte_9  payload_byte_10  ...  payload_byte_1496  payload_byte_1497  \
0       81.285714        77.714286  ...                0.0                0.0   
1       

In [59]:
df

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,payload_byte_1496,payload_byte_1497,payload_byte_1498,payload_byte_1499,payload_byte_1500,ttl,total_len,t_delta,0,0.1
0,102.285714,174.000000,99.142857,66.000000,82.714286,79.142857,103.571429,98.571429,81.285714,77.714286,...,0.0,0.0,0.0,0.0,0.0,0.0,858.285714,0.000015,-1.0,normal
1,82.714286,131.142857,51.000000,82.000000,54.714286,97.285714,74.571429,96.714286,91.571429,65.285714,...,0.0,0.0,0.0,0.0,0.0,0.0,465.714286,0.008597,-1.0,normal
2,31.285714,106.571429,129.000000,131.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,60.0,132.000000,0.000006,17.0,normal
3,112.571429,0.428571,129.000000,131.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,60.0,132.000000,0.000006,17.0,normal
4,18.571429,138.142857,129.000000,131.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,60.0,132.000000,0.000005,17.0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,86.142857,88.285714,81.571429,76.428571,70.857143,92.285714,85.857143,95.857143,87.285714,81.857143,...,0.0,0.0,0.0,0.0,0.0,254.0,1447.000000,0.000006,6.0,exploits
217,182.142857,182.142857,186.714286,215.142857,255.000000,205.857143,204.142857,201.000000,214.857143,182.142857,...,0.0,0.0,0.0,0.0,0.0,254.0,1244.428571,0.000006,6.0,fuzzers
218,68.714286,71.428571,84.142857,72.714286,42.000000,65.714286,71.000000,57.857143,57.714286,59.285714,...,0.0,0.0,0.0,0.0,0.0,62.0,683.571429,0.000011,6.0,exploits
219,75.142857,73.857143,73.000000,81.142857,41.285714,60.285714,56.428571,52.857143,50.571429,51.857143,...,0.0,0.0,0.0,0.0,0.0,62.0,367.857143,0.000008,6.0,exploits


In [60]:
# rename the column 'A' to 'X'
df = df.rename(columns={'0': 'protocol_m','0.1': 'label'})

In [61]:
df

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,payload_byte_1496,payload_byte_1497,payload_byte_1498,payload_byte_1499,payload_byte_1500,ttl,total_len,t_delta,protocol_m,label
0,102.285714,174.000000,99.142857,66.000000,82.714286,79.142857,103.571429,98.571429,81.285714,77.714286,...,0.0,0.0,0.0,0.0,0.0,0.0,858.285714,0.000015,-1.0,normal
1,82.714286,131.142857,51.000000,82.000000,54.714286,97.285714,74.571429,96.714286,91.571429,65.285714,...,0.0,0.0,0.0,0.0,0.0,0.0,465.714286,0.008597,-1.0,normal
2,31.285714,106.571429,129.000000,131.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,60.0,132.000000,0.000006,17.0,normal
3,112.571429,0.428571,129.000000,131.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,60.0,132.000000,0.000006,17.0,normal
4,18.571429,138.142857,129.000000,131.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,60.0,132.000000,0.000005,17.0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,86.142857,88.285714,81.571429,76.428571,70.857143,92.285714,85.857143,95.857143,87.285714,81.857143,...,0.0,0.0,0.0,0.0,0.0,254.0,1447.000000,0.000006,6.0,exploits
217,182.142857,182.142857,186.714286,215.142857,255.000000,205.857143,204.142857,201.000000,214.857143,182.142857,...,0.0,0.0,0.0,0.0,0.0,254.0,1244.428571,0.000006,6.0,fuzzers
218,68.714286,71.428571,84.142857,72.714286,42.000000,65.714286,71.000000,57.857143,57.714286,59.285714,...,0.0,0.0,0.0,0.0,0.0,62.0,683.571429,0.000011,6.0,exploits
219,75.142857,73.857143,73.000000,81.142857,41.285714,60.285714,56.428571,52.857143,50.571429,51.857143,...,0.0,0.0,0.0,0.0,0.0,62.0,367.857143,0.000008,6.0,exploits


In [62]:
# save the new dataframe to a new csv file
df.to_csv('UNSW-7-number-packets-to-flows.csv', index=False)


In [63]:
df

,payload_byte_1,payload_byte_2,payload_byte_3,payload_byte_4,payload_byte_5,payload_byte_6,payload_byte_7,payload_byte_8,payload_byte_9,payload_byte_10,...,payload_byte_1496,payload_byte_1497,payload_byte_1498,payload_byte_1499,payload_byte_1500,ttl,total_len,t_delta,protocol_m,label
0,102.285714,174.000000,99.142857,66.000000,82.714286,79.142857,103.571429,98.571429,81.285714,77.714286,...,0.0,0.0,0.0,0.0,0.0,0.0,858.285714,0.000015,-1.0,normal
1,82.714286,131.142857,51.000000,82.000000,54.714286,97.285714,74.571429,96.714286,91.571429,65.285714,...,0.0,0.0,0.0,0.0,0.0,0.0,465.714286,0.008597,-1.0,normal
2,31.285714,106.571429,129.000000,131.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,60.0,132.000000,0.000006,17.0,normal
3,112.571429,0.428571,129.000000,131.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,60.0,132.000000,0.000006,17.0,normal
4,18.571429,138.142857,129.000000,131.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,60.0,132.000000,0.000005,17.0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,86.142857,88.285714,81.571429,76.428571,70.857143,92.285714,85.857143,95.857143,87.285714,81.857143,...,0.0,0.0,0.0,0.0,0.0,254.0,1447.000000,0.000006,6.0,exploits
217,182.142857,182.142857,186.714286,215.142857,255.000000,205.857143,204.142857,201.000000,214.857143,182.142857,...,0.0,0.0,0.0,0.0,0.0,254.0,1244.428571,0.000006,6.0,fuzzers
218,68.714286,71.428571,84.142857,72.714286,42.000000,65.714286,71.000000,57.857143,57.714286,59.285714,...,0.0,0.0,0.0,0.0,0.0,62.0,683.571429,0.000011,6.0,exploits
219,75.142857,73.857143,73.000000,81.142857,41.285714,60.285714,56.428571,52.857143,50.571429,51.857143,...,0.0,0.0,0.0,0.0,0.0,62.0,367.857143,0.000008,6.0,exploits


In [64]:
df.label.value_counts()

exploits    82
analysis    73
fuzzers     26
generic     19
dos          9
normal       6
backdoor     6
Name: label, dtype: int64